# Object Detection Starter — Colab/Kaggle Quickstart

> Author : Badr TAJINI

**Academic year:** 2025–2026  
**School:** ECE  
**Course:** Machine Learning & Deep Learning 2 

---

Follow this guide to validate the YOLOv8 starter on a free GPU runtime. You will confirm the environment, install dependencies, run a smoke prediction, and optionally fine-tune/evaluate on coco128.

## 0. Project files

- Option A: `git clone <your_repo_url> od-project`
- Option B: Upload the `od-project` folder via the sidebar (ensure it appears as `/content/od-project`).

Run the next cells afterwards; they will error helpfully if the folder is missing.

### Quick checklist before running code
- Switch the runtime to **GPU (T4)** first.
  - Colab: `Runtime → Change runtime type → GPU → Save`.
  - Kaggle: gear icon → enable **Accelerator**, choose `T4 x1`.
- Wait for the runtime to reconnect and show a GPU badge.
- Execute cells from top to bottom. If a cell fails, fix it and rerun that cell before moving on.

### How to run a cell
- Click the ▶️ button to the left of a cell, or press **Shift+Enter** (Colab) / **Ctrl+Enter** (Kaggle).
- When a cell finishes, the number on the left becomes `[1]`, `[2]`, etc.
- Don’t skip steps—the later ones depend on the earlier setup.

### Step 0 — Confirm the GPU is ready
Run the next cell. You should see GPU name and memory. If you see `nvidia-smi unavailable`, the runtime is still on CPU—return to the checklist, enable GPU, then rerun this cell.

In [ ]:
!nvidia-smi || echo "nvidia-smi unavailable (CPU runtime)"


### Step 1 — Point the notebook at the project folder
This cell ensures the current working directory is `od-project`.
If it raises a `FileNotFoundError`, confirm where you uploaded/cloned the folder and adjust the path before rerunning.

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
elif PROJECT_ROOT.name == "content":
    candidate = PROJECT_ROOT / "od-project"
    if candidate.exists():
        PROJECT_ROOT = candidate.resolve()

if not (PROJECT_ROOT / "src").exists():
    raise FileNotFoundError(
        f"Could not locate project root at {PROJECT_ROOT}. Upload or clone od-project before proceeding."
    )

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))
print(f"Project root: {PROJECT_ROOT}")


### Step 2 — Install the project requirements
Installs Ultralytics + supporting libraries from `requirements.txt`. First run downloads the YOLO packages, which may take a couple of minutes.
If installation fails (e.g., due to connectivity), rerun this cell before proceeding.

In [ ]:
# Install project dependencies listed in requirements.txt
!pip install -r requirements.txt


### Step 3 — Run the smoke test
This loads the pretrained YOLO weights specified in the config and runs a single prediction on a sample image. It writes `outputs/smoke_metrics.json` so you know the toolkit is ready.
If you hit a download error, wait briefly and rerun the cell.

In [ ]:
from src import smoke_check

smoke_path = smoke_check.run_smoke("configs/yolo_coco128.yaml")
print(smoke_path.read_text())


## 1. Review smoke-test output
- Confirm the previous cell printed a JSON block showing the number of detections and mean confidence.
- `outputs/smoke_metrics.json` should now appear in the file browser.
- Need only a quick environment check? Stop here. Ready to fine-tune? Continue to Section 2.
- If anything failed, read the message, fix the issue, and rerun the smoke cell before moving on.

## 2. Full training run (optional)
Run these cells when you want to fine-tune YOLOv8 on `coco128` (or your own dataset). On the first run, downloading the dataset weights can take a couple of minutes.

**Before running:**
1. Open `configs/yolo_coco128.yaml` if you want to edit epochs, batch size, or dataset paths.
2. Ensure the runtime still shows a GPU connection.
3. Close other heavy tabs so the GPU memory stays available.

In [ ]:
# Train the model using the default coco128 configuration.
# Ultralytics manages the training loop and saves weights to outputs/train/weights/.
!python src/train.py --config configs/yolo_coco128.yaml


In [ ]:
# Evaluate the best checkpoint on the validation split.
# Results (mAP, precision, recall) are saved to outputs/eval.json.
!python src/evaluate.py --config configs/yolo_coco128.yaml --ckpt outputs/train/weights/best.pt


### Step 4 — What should I see now?
- `outputs/train/weights/best.pt`: best fine-tuned weights.
- `outputs/train/results.csv` and `outputs/train/events/`: Ultralytics training logs.
- `outputs/eval.json`: validation metrics (mAP50, mAP, precision, recall).
- Optional: run `python src/predict.py ...` to generate annotated images.
If any files are missing, scroll up for errors in the training/evaluation cells.

## 3. Mirror this workflow for custom datasets
1. Duplicate this notebook for your dataset (e.g., `01_custom_yolo.ipynb`).
2. Update the config file (e.g., `configs/yolo_custom.yaml`) to point at your `data.yaml`.
3. Swap the train/eval commands to use the custom config and checkpoint paths.
4. Use `python src/predict.py --config <cfg> --ckpt <weights> --source <images>` to test the trained model.

Keep the order (GPU check → install → smoke test → train → evaluate) for reliable, reproducible runs.